# Drive characteristics using `strym`

In this notebook, we will generate characteristics of a particular drive. Please ensure strym is installed correctly

## Installing `strym`
First, we are required to install `strym`. For that, we will just use github devtools functionality.
Even if you have installed `strym` previously, you should reinstall for latest version of the package.

In [67]:
# Installing strym using pip from git
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/jmscslgroup/strym

  Cloning https://github.com/jmscslgroup/strym to /private/var/folders/qb/xzd2v2fn741b459s8jvfjhk4000zl1/T/pip-req-build-beww1_nv
  Running command git clone -q https://github.com/jmscslgroup/strym /private/var/folders/qb/xzd2v2fn741b459s8jvfjhk4000zl1/T/pip-req-build-beww1_nv
  Created wheel for strym: filename=strym-0.1-cp37-none-any.whl size=19497 sha256=5d2307695f1b9342691c1e877220c65b4fb504a81866091fd27fed40277af038
  Stored in directory: /private/var/folders/qb/xzd2v2fn741b459s8jvfjhk4000zl1/T/pip-ephem-wheel-cache-en2ca1es/wheels/16/8c/7f/7ab5f7b26c83fdbe3972de24a6aeb113cf4a35c9125d065c9d
Successfully built strym
  Found existing installation: strym 0.1
    Uninstalling strym-0.1:
      Successfully uninstalled strym-0.1


## Importing packages
Import required packages

In [97]:
from strym import strymread
import strym
import math
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate

## Specify Data Location

In [229]:
datafolder = "/Users/sprinkle/work/research/circles/cyverse/2020_03_03/"
import glob
csvlist = glob.glob(datafolder+"*.csv")

In [230]:
num_of_files = len(csvlist)
print("Total number of datafiles in {} is {}.".format(datafolder, num_of_files))

Total number of datafiles in /Users/sprinkle/work/research/circles/cyverse/2020_03_03/ is 7.


## Analysis
### 1. CSV file containing all messages
In this section, we will extract a subset of the CSV-formatted CAN Data in order to produce summary information of a specific drive.

In [239]:
dbcfile = '../examples/newToyotacode.dbc'
for i in range(num_of_files):
    r0 = strymread(csvfile=csvlist[i], dbcfile=dbcfile)
    print(f'Reading {csvlist[i]}')
    duration1 = r0.dataframe['Time'][len(r0.dataframe)-1] - r0.dataframe['Time'][0]
    duration_str = f'  Duration of this drive is {duration1} seconds ({math.trunc(int(duration1*1000)/(1000*60))} minutes {math.trunc(duration1 % 60)} seconds).'
    start_str = f'  Starting date/time of the drive is {time.ctime(r0.dataframe["Time"][0])}'

    # get the speed timeseries information from the data file
    speed_ts = r0.ts_speed()
    # turn the timeseries into a python array for integration
    # transform from km/hr by multiplying 1000m/1km and 1 hr/3600s to get m/s
    speed_ar = np.array(speed_ts['Message'])*1000/3600
#     speed_ar[0:-1]*1000/3600
    # find the difference of the time values
#     dt = np.diff(np.array(speed_ts['Time']))
    dt = np.diff(speed_ts['Time'])
    # trapezoidal integration, divide by 1000 to get total km (rather than m)
    # Commented out: this produces negative values that don't make sense
#     km_ts = np.trapz(y=speed_ar[0:-1],x=dt)/1000
    km_ts = np.trapz(y=speed_ar,x=np.array(speed_ts['Time']))/1000
    # commented out: this produces incorrect values that seem to be off by a factor of around 2, depending
    km_dx = np.trapz(y=speed_ar,dx=0.02)/1000
    # need to convert km/hr to km/s to get km later when integrating
    distance_str = f'  The trip was {km_ts} km ({km_ts*3.1/5} miles)'
#     distance_str_dx f'  The dx version is {km_dx} km ({km_dx*3.1/5} miles)'

    #speed_ts['Time']
    #f'Total miles driven is {km_ts}'
    print(duration_str)
    print(start_str)
    print(distance_str)


Reading /Users/sprinkle/work/research/circles/cyverse/2020_03_03/2020-03-03-19-32-39-704415__CAN_Messages.csv
  Duration of this drive is 1306.5440361499786 seconds (21 minutes 46 seconds).
  Starting date/time of the drive is Tue Mar  3 19:32:40 2020
  The trip was 8.649056945562362 km (5.362415306248665 miles)
Reading /Users/sprinkle/work/research/circles/cyverse/2020_03_03/2020-03-03-10-35-13-966077__CAN_Messages.csv
  Duration of this drive is 2712.505082845688 seconds (45 minutes 12 seconds).
  Starting date/time of the drive is Tue Mar  3 10:35:14 2020
  The trip was 30.322010043714446 km (18.799646227102958 miles)
Reading /Users/sprinkle/work/research/circles/cyverse/2020_03_03/2020-03-03-19-57-11-286348__CAN_Messages.csv
  Duration of this drive is 1022.5458858013153 seconds (17 minutes 2 seconds).
  Starting date/time of the drive is Tue Mar  3 19:57:11 2020
  The trip was 14.35129696881142 km (8.89780412066308 miles)
Reading /Users/sprinkle/work/research/circles/cyverse/2020_